In [ ]:
import os 
import sys 
# from funcs.kiwoom.pykiwoom import *

from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
import pythoncom
import win32gui
import win32con
import win32api
import time
import multiprocessing as mp


#--------------------------------------------------------------------
# 자동 로그인 해제 및 설정
#--------------------------------------------------------------------
LOGIN_FILE     = "C:/OpenAPI/system/Autologin.dat"
LOGIN_FILE_TMP = "C:/OpenAPI/system/Autologin.tmp"


def turn_off_auto():
    if os.path.isfile(LOGIN_FILE):
        os.rename(LOGIN_FILE, LOGIN_FILE_TMP) 

def turn_on_auto():
    if os.path.isfile(LOGIN_FILE_TMP):
        os.rename(LOGIN_FILE_TMP, LOGIN_FILE)


#--------------------------------------------------------------------
# 수동 로그인 관련 함수
#--------------------------------------------------------------------
def window_enumeration_handler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))


def enum_windows():
    windows = []
    win32gui.EnumWindows(window_enumeration_handler, windows)
    return windows


def find_window(caption):
    hwnd = win32gui.FindWindow(None, caption)
    if hwnd == 0:
        windows = enum_windows()
        for handle, title in windows:
            if caption in title:
                hwnd = handle
                break
    return hwnd


def enter_keys(hwnd, data, interval=500):
    win32api.SendMessage(hwnd, win32con.EM_SETSEL, 0, -1)
    win32api.SendMessage(hwnd, win32con.EM_REPLACESEL, 0, data)
    win32api.Sleep(interval)


def click_button(btn_hwnd):
    win32api.PostMessage(btn_hwnd, win32con.WM_LBUTTONDOWN, 0, 0)
    win32api.Sleep(100)
    win32api.PostMessage(btn_hwnd, win32con.WM_LBUTTONUP, 0, 0)
    win32api.Sleep(300)


def left_click(x, y, hwnd):
    lParam = win32api.MAKELONG(x, y)
    win32gui.SendMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, lParam)
    win32gui.SendMessage(hwnd, win32con.WM_LBUTTONUP, 0, lParam)


def double_click(x, y, hwnd):
    left_click(x, y, hwnd)
    left_click(x, y, hwnd)
    win32api.Sleep(300)


#--------------------------------------------------------------------
# 로그인창
#--------------------------------------------------------------------
class MyWindow(QWidget):
    app = QApplication(sys.argv)

    def __init__(self, q):
        super().__init__()
        self.login_status = False
        self.q = q
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self.slot_login)
        self.login()

    def login(self):
        self.ocx.dynamicCall("CommConnect()")
        while not self.login_status: 
            pythoncom.PumpWaitingMessages()
            time.sleep(0.001)

    def slot_login(self, err_code):           
        if err_code == 0:
            self.KOA_Functions(_T("ShowAccountWindow"), _T("")) 
        self.login_status = True
        self.q.put('complete')


def version(user_id, user_pass, user_cert=None, acc_pw=None):
    q = mp.Queue()

    # 자동 로그인 해제
    turn_off_auto()

    sub_proc = mp.Process(target=MyWindow, name="Sub Process", args=(q,), daemon=True)
    sub_proc.start()

    while True:
        caption = "Open API Login"
        hwnd = find_window(caption)

        if hwnd == 0:
            print("로그인 창 대기 ...")
            time.sleep(1)
            continue
        else:
            break

    time.sleep(2)
    edit_id   = win32gui.GetDlgItem(hwnd, 0x3E8)
    edit_pass = win32gui.GetDlgItem(hwnd, 0x3E9)
    edit_cert = win32gui.GetDlgItem(hwnd, 0x3EA)
    btn_login = win32gui.GetDlgItem(hwnd, 0x1)

    if user_cert is None:
        if win32gui.IsWindowEnabled(win32gui.GetDlgItem(hwnd, 0x3EA)):
            click_button(win32gui.GetDlgItem(hwnd, 0x3ED))
    else:
        if not win32gui.IsWindowEnabled(win32gui.GetDlgItem(hwnd, 0x3EA)):
            click_button(win32gui.GetDlgItem(hwnd, 0x3ED))

    double_click(15, 15, edit_id)
    enter_keys(edit_id, user_id) 
    time.sleep(0.5)

    double_click(15, 15, edit_pass)
    enter_keys(edit_pass, user_pass) 
    time.sleep(0.5)

    if user_cert is not None:
        double_click(15, 15, edit_cert)
        enter_keys(edit_cert, user_cert) 
        time.sleep(0.5)

    # double_click(15, 15, edit_id)
    # enter_keys(edit_id, user_id) 
    # time.sleep(1)
    click_button(btn_login)   

    secs_cnt = 0
    while True:
        time.sleep(1)
        remain_secs = 120 - secs_cnt
        print(f"로그인 대기: {remain_secs}")

        # 버전처리 경고창 확인
        hwnd = find_window("opstarter")
        if hwnd != 0:
            try:
                static_hwnd = win32gui.GetDlgItem(hwnd, 0xFFFF)
                text = win32gui.GetWindowText(static_hwnd)
                if '버전처리' in text:
                    while sub_proc.is_alive():
                        sub_proc.kill()
                        time.sleep(1)

                    click_button(win32gui.GetDlgItem(hwnd, 0x2))
                    secs_cnt = 90      # 버전처리이면 30초 후 종료
            except:
                pass

        # 업그레이드 확인창
        hwnd = find_window("업그레이드 확인")
        if hwnd != 0:
            win32gui.PostMessage(hwnd, win32con.WM_CLOSE, 0, 0)

        # 버전처리가 있는 경우의 종료
        if secs_cnt > 120:
            break

        # 계좌비밀번호 입력창 확인
        hwnd = find_window("계좌비밀번호 입력")
        print("hwnd :", hwnd)
        if hwnd != 0:            
            acc_pw_pane   = win32gui.GetDlgItem(hwnd, 0xCC)
            enlist_bt = win32gui.GetDlgItem(hwnd, 0xCF)
            auto_bt = win32gui.GetDlgItem(hwnd, 0xD3)
            # close_bt = win32gui.GetDlgItem(hwnd, 0x1)
            
            double_click(15, 15, acc_pw_pane)
            enter_keys(acc_pw_pane, acc_pw) 
            time.sleep(0.5)
            
            click_button(auto_bt)  
            time.sleep(0.5)
            
            click_button(enlist_bt)   
            
            win32gui.PostMessage(hwnd, win32con.WM_CLOSE, 0, 0)
        
        # 버전처리가 없는 경우의 종료
        if not q.empty():
            data = q.get()
            break

        secs_cnt += 1
    
    print("버전처리 완료")

    # 자동 로그인 재설정
    turn_on_auto()


if __name__ == "__main__":
    version("nave94", "jayone89", "jayone8974^^", "8974")

In [1]:
import sys 
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
import pythoncom
import time
import multiprocessing as mp

class MyWindow(QWidget):
    app = QApplication(sys.argv)

    def __init__(self, q):
        super().__init__()
        self.login_status = False
        self.q = q
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self.slot_login)
        self.login()

    def login(self):
        self.ocx.dynamicCall("CommConnect()")
        while not self.login_status: 
            pythoncom.PumpWaitingMessages()
            time.sleep(0.001)

    def slot_login(self, err_code):           
        if err_code == 0:
            self.ocx.KOA_Functions("ShowAccountWindow", "")
        self.login_status = True
        self.q.put('complete')

#     def __init__(self):
#         super().__init__()
#         self.login_status = False
#         self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
#         self.ocx.OnEventConnect.connect(self.slot_login)
#         self.login()

#     def login(self):
#         self.ocx.dynamicCall("CommConnect()")
#         while not self.login_status: 
#             pythoncom.PumpWaitingMessages()
#             time.sleep(0.001)

#     def slot_login(self, err_code):   
#         if err_code == 0: 
#             self.ocx.KOA_Functions("ShowAccountWindow", "")
#         self.login_status = True
#         print("login is complete")


if __name__ == "__main__":
    
    # window = MyWindow()
    # app.exec_()        
    
    # sub_proc = mp.Process(target=MyWindow, name="Sub Process")    
    # sub_proc.start()
    
    q = mp.Queue()
    # procs = []
    
    # sub_proc = mp.Process(target=MyWindow, name="Sub Process", args=(q,), daemon=True)
    sub_proc = mp.Process(target=MyWindow, name="Sub Process", args=(q,))
    sub_proc.start()
    sub_proc.join()

    # for i in range(60):
    #     print("sleep ", 60-i)
    #     time.sleep(1)

#     if sub_proc.is_alive():
#         sub_proc.kill()

#     while True:
#         print("main process is running")
#         time.sleep(1)

# multi_login reference

In [ ]:
from threading import Thread
from multiprocessing import Process, Lock
import time
import sys


class Kiwoom(QWidget):
    
    def __init__(self):
        super().__init__()
        self.login_status = False
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self.slot_login)
        # self.login()

    def login(self):
        self.ocx.dynamicCall("CommConnect()")
        while not self.login_status: 
            pythoncom.PumpWaitingMessages()
            time.sleep(0.001)

    def slot_login(self, err_code):   
        # if err_code == 0: 
        #     self.ocx.KOA_Functions("ShowAccountWindow", "")
        self.login_status = True
        print("login is complete")
 
def login_input(user_id, norm_pwd, cert_pwd, is_simulated=False):
    import pywinauto
    # sys.coinit_flags=0
    while True:
        procs = pywinauto.findwindows.find_elements()
        for proc in procs:
            if proc.name == 'Open API Login':
                break
        if proc.name == 'Open API Login':
            break
 
    login_app = pywinauto.Application().connect(process=proc.process_id)
    login_dig = login_app.OpenAPILogin
    login_dig.Edit1.send_keystrokes(user_id)
    login_dig.Edit2.send_keystrokes(norm_pwd)
    if is_simulated:
        if login_dig.Edit3.is_enabled():
            login_dig.Button5.click()  # check mock invest server mode
        login_dig.Edit2.send_keystrokes('{ENTER}') # login
    else:
        if not login_dig.Edit3.is_enabled():
            login_dig.Button5.click()  # uncheck mock invest server mode
        login_dig.Edit3.send_keystrokes(cert_pwd)
        login_dig.Edit3.send_keystrokes('{ENTER}') # login
 
def multi(idx, lock, id, pwd, cert):
    app = QApplication(sys.argv)
    kiwoom = Kiwoom()
    
    print("work")
 
    if idx % 2 == 1: # check mock invest server mode
        is_simulated = True
    else:
        is_simulated = False
    lock.acquire() # process lock
    login_th = Thread(target=login_input,
                      kwargs={'user_id':id,
                             'norm_pwd':pwd,
                             'cert_pwd':cert,
                             'is_simulated':is_simulated})
    login_th.start()
    kiwoom.login()
    lock.release() # process unlock
 
    print('process[{0}]'.format(idx))
    print('Login state : {0}'.format(kiwoom.get_connect_state()))
    print('Server : {0}'.format(kiwoom.get_login_info("GetServerGubun")))
    #app.exec_() # not to terminate process
 
if __name__=='__main__':
    
    procs = []
    lock = Lock()
    
    
    multi(0, lock, "nave94", "jayone89", "jayone8974{^}{^}")
    
    # start_time = time.time()
    # for idx in range(2):
    #     proc = Process(target=multi, args=(idx, lock, "nave94", "jayone89", "jayone8974{^}{^}"))
    #     procs.append(proc)
    #     proc.start()
    # for proc in procs:
    #     proc.join()
    # end_time = time.time()
    # print('time : {0}'.format(end_time - start_time))

In [ ]:
1

# custom3

In [9]:
from threading import Thread
from multiprocessing import Process, Lock
import os 
import sys 
# from funcs.kiwoom.pykiwoom import *

from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
import pythoncom
import win32gui
import win32con
import win32api
import time

class Kiwoom(QWidget):
    
    def __init__(self):
        super().__init__()
        self.login_status = False
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self.slot_login)
        # self.login()

    def login(self):
        self.ocx.dynamicCall("CommConnect()")
        while not self.login_status: 
            pythoncom.PumpWaitingMessages()
            time.sleep(0.001)

    def slot_login(self, err_code):   
        # if err_code == 0: 
        #     self.ocx.KOA_Functions("ShowAccountWindow", "")
        self.login_status = True
        print("login is complete")
        
def func(name, idx, lock):
    app = QApplication(sys.argv)
    kiwoom = Kiwoom()
    
    print("work")
    
    id = "nave94"
    pwd = "jayone89"
    cert = "jayone8974^^"
 
    if idx % 2 == 1: # check mock invest server mode
        is_simulated = True
    else:
        is_simulated = False
    lock.acquire() # process lock
    login_th = Thread(target=login_input,
                      kwargs={'user_id':id,
                             'norm_pwd':pwd,
                             'cert_pwd':cert,
                             'is_simulated':is_simulated})
    login_th.start()
    kiwoom.login()
    lock.release() # process unlock
 
    print('process[{0}]'.format(idx))
    print('Login state : {0}'.format(kiwoom.get_connect_state()))
    print('Server : {0}'.format(kiwoom.get_login_info("GetServerGubun")))
    
    start = time.time()
    print(name, " started")
    time.sleep(2)
    delta_t = time.time()-start
    print(name, " ended, took ",delta_t)

if __name__ == "__main__":
    
    lock = Lock()
    # Without Process
    # start = time.time()
    # func("No Process 1", 0, lock)
    # func("No Process 2", 0, lock)
    # delta_t = time.time() - start
    # print("Without Process : ",delta_t,"s")
    
    # With Process
    start = time.time()
    proc1 = Process(target=func,args=('process1', 0, lock,))
    proc1.start()
    proc1.join()
    delta_t = time.time()- start
    print("With Process : ",delta_t,"s")

With Process :  0.07489466667175293 s
